In [1]:
#Question 1

import json
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.api as sm
import scipy as sc
from scipy.stats import mode
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import Normalizer,StandardScaler

en_countries = ['au','ca', 'gb', 'nz', 'us']
 
twit_id = 333903271
candy_id = 553834731
facebook_id = 284882215
tinder_id = 547702041

page_list = [1,2]

def app(id):
  df_total = pd.DataFrame(columns = ['im:version','im:rating', 'title', 'content'])
  for lang in en_countries:
    for page in page_list:
      data = requests.get(f"https://itunes.apple.com/{lang}/rss/customerreviews/id={id}/page={page}/sortby=mostrecent/json")
      d = json.loads(data.content)
      df = pd.DataFrame(d['feed']['entry'])
      df['title'] = df['title'].apply(lambda x: x['label'])
      df['im:version'] = df['im:version'].apply(lambda x: x['label'])
      df['im:rating'] = df['im:rating'].apply(lambda x: x['label'])
      df['content'] = df['content'].apply(lambda x: x['label'])
      df = df.drop(['author', 'id', 'link', 'im:voteSum', 'im:contentType', 'im:voteCount'], axis = 1)
      df_total = pd.concat([df_total, df], axis = 0)
  return df_total

twitter = app(333903271)#program kept crashing after it was already working. Only started working with number instead of twit_id
candy = app(candy_id)
facebook = app(facebook_id)
tinder = app(tinder_id)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
twitter['App'] = twitter.apply(lambda x: 'Twitter', axis = 1)
twitter

,im:version,im:rating,title,content,App
0,8.56,2,bad app. wont let me make a space. i want to b...,wanna make a space,Twitter
1,8.55.1,3,new feature spaces,the new feature “spaces” is causing a lot of a...,Twitter
2,8.55.1,5,WOW,I love a lot of Twitter account....I spent lot...,Twitter
3,8.55,2,Voice notes,Why for some of us voice notes are not available,Twitter
4,8.55,2,Censorship,Censorship will lead to dictatorship. Even if ...,Twitter
...,...,...,...,...,...
45,8.55.1,1,Locked out,"I haven’t been able to engage(like, comment, t...",Twitter
46,8.55.1,1,Sell no ad version,Let me pay $1/mo or whatever not to see your t...,Twitter
47,8.55.1,1,Garbage,Garbage anti free speech Pravda app is garbage...,Twitter
48,8.55.1,5,Frustrated person.,I’m sorry you don’t like when someone speaks t...,Twitter


In [3]:
candy['App'] = candy.apply(lambda x: 'Candy Crush', axis = 1)
candy

,im:version,im:rating,title,content,App
0,1.197.1.1,1,Can’t open since the new update on 8/3/21,Disappointed this new update has caused a glit...,Candy Crush
1,1.197.1.1,1,New update,New update won’t work at all. I can’t play it ...,Candy Crush
2,1.197.1.1,2,Meh,Its a good game but youve made it impossible t...,Candy Crush
3,1.197.0.1,5,So pleased,You guys are amazing! The creator of this game...,Candy Crush
4,1.197.0.1,3,Top of play zone cut off with new iPhone.,I just got a new iPhone 12 mini and I can’t se...,Candy Crush
...,...,...,...,...,...
45,1.197.0.1,2,Game was addicting..... Until now,"This game did almost everything right, kept yo...",Candy Crush
46,1.196.1.1,5,Meditation,Playing the game is like a meditation for me,Candy Crush
47,1.197.0.1,5,Sugar crush,Truly a great go to for me during my “me time”,Candy Crush
48,1.197.0.1,1,Still won’t load,"This is week 4, update 4, and the app still wo...",Candy Crush


In [4]:
facebook['App'] = facebook.apply(lambda x: 'Facebook', axis = 1)
facebook

,im:version,im:rating,title,content,App
0,308.0,1,CANT LOG IN,This dog sh*** app won’t let me log into my ac...,Facebook
1,308.0,1,Far too many ads,I can no longer use this app. The app keeps co...,Facebook
2,308.0,1,Unexpected error please try login again.,Why does it keep saying Unexpected error pleas...,Facebook
3,308.0,1,Request for review,You guys deleted my 9 years old account for no...,Facebook
4,308.0,2,Marketplace crashing,"I have done all the updates, turned off and on...",Facebook
...,...,...,...,...,...
45,308.0,1,Frightening privacy problems,Read the privacy labels. You’re giving them ev...,Facebook
46,308.0,1,Ruined marketplace.,Facebook has to be one of the most tone deaf c...,Facebook
47,308.0,1,Toxic,"Social media is poisonous, & Facebook just fue...",Facebook
48,308.0,1,Still unable to create avatar,App keeps telling me I don’t have the latest v...,Facebook


In [5]:
tinder['App'] = tinder.apply(lambda x: 'Tinder', axis = 1)
tinder

,im:version,im:rating,title,content,App
0,12.3.0,1,Joke!!!,Overcharged charged more than once can never g...,Tinder
1,12.3.0,1,Disgusted,My account got banned as soon as I switched my...,Tinder
2,12.2.0,1,Full of scammers,I used to like Tinder but now almost every sec...,Tinder
3,12.3.0,2,Fix the app asap,Please fix!! The amount of times I’ve had to u...,Tinder
4,12.3.0,1,Bugs,"I pay for tinder and all I get is bugs, glitch...",Tinder
...,...,...,...,...,...
45,12.3.0,1,Fake profiles,Giant waste of time. Canceling subscription.,Tinder
46,12.3.0,5,Shmoopy,I met the shmoop of my life on this app. 10/10...,Tinder
47,12.3.0,1,Hate it!,Completely worthless!,Tinder
48,12.3.0,1,I don’t like it at all,I hate it,Tinder


In [6]:
data = pd.concat((twitter, candy, facebook, tinder))
data

,im:version,im:rating,title,content,App
0,8.56,2,bad app. wont let me make a space. i want to b...,wanna make a space,Twitter
1,8.55.1,3,new feature spaces,the new feature “spaces” is causing a lot of a...,Twitter
2,8.55.1,5,WOW,I love a lot of Twitter account....I spent lot...,Twitter
3,8.55,2,Voice notes,Why for some of us voice notes are not available,Twitter
4,8.55,2,Censorship,Censorship will lead to dictatorship. Even if ...,Twitter
...,...,...,...,...,...
45,12.3.0,1,Fake profiles,Giant waste of time. Canceling subscription.,Tinder
46,12.3.0,5,Shmoopy,I met the shmoop of my life on this app. 10/10...,Tinder
47,12.3.0,1,Hate it!,Completely worthless!,Tinder
48,12.3.0,1,I don’t like it at all,I hate it,Tinder


In [7]:
COMPRESSED_SIZE = 200

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['content'])

Xd = X.toarray()
Xd = PCA(COMPRESSED_SIZE).fit(Xd.T).components_.T
Xd = sm.add_constant(Xd)

In [8]:
k = 5
model = KMeans(n_clusters = k, init = 'k-means++', max_iter = 100, n_init = 1)
model.fit(X)
clusters = model.predict(X)

In [9]:
data['im:rating'] = data['im:rating'].astype(int)
y = np.array(data['im:rating'])
labels = np.zeros_like(clusters)
for i in range(5):
    mask = (clusters == i)
    labels[clusters == i] = mode(y[clusters == i])[0]

In [10]:
data['kmeans_tfidf'] = labels
data

,im:version,im:rating,title,content,App,kmeans_tfidf
0,8.56,2,bad app. wont let me make a space. i want to b...,wanna make a space,Twitter,1
1,8.55.1,3,new feature spaces,the new feature “spaces” is causing a lot of a...,Twitter,1
2,8.55.1,5,WOW,I love a lot of Twitter account....I spent lot...,Twitter,1
3,8.55,2,Voice notes,Why for some of us voice notes are not available,Twitter,1
4,8.55,2,Censorship,Censorship will lead to dictatorship. Even if ...,Twitter,1
...,...,...,...,...,...,...
45,12.3.0,1,Fake profiles,Giant waste of time. Canceling subscription.,Tinder,1
46,12.3.0,5,Shmoopy,I met the shmoop of my life on this app. 10/10...,Tinder,1
47,12.3.0,1,Hate it!,Completely worthless!,Tinder,1
48,12.3.0,1,I don’t like it at all,I hate it,Tinder,1


In [11]:
fowlkes_mallows_score(data.kmeans_tfidf, y)

0.6318470828485205

In [12]:
accuracy_score(data.kmeans_tfidf, y)

0.6265

In [13]:
r2_score(data['im:rating'], data.kmeans_tfidf)

-0.343428372923821

In [14]:
#Question 2

import gensim

data.content = data.content.astype(str)
data.content = [x.lower() for x in data.content]
data

,im:version,im:rating,title,content,App,kmeans_tfidf
0,8.56,2,bad app. wont let me make a space. i want to b...,wanna make a space,Twitter,1
1,8.55.1,3,new feature spaces,the new feature “spaces” is causing a lot of a...,Twitter,1
2,8.55.1,5,WOW,i love a lot of twitter account....i spent lot...,Twitter,1
3,8.55,2,Voice notes,why for some of us voice notes are not available,Twitter,1
4,8.55,2,Censorship,censorship will lead to dictatorship. even if ...,Twitter,1
...,...,...,...,...,...,...
45,12.3.0,1,Fake profiles,giant waste of time. canceling subscription.,Tinder,1
46,12.3.0,5,Shmoopy,i met the shmoop of my life on this app. 10/10...,Tinder,1
47,12.3.0,1,Hate it!,completely worthless!,Tinder,1
48,12.3.0,1,I don’t like it at all,i hate it,Tinder,1


In [15]:
words = data.content.str.split()
words = pd.DataFrame(words.tolist())

In [16]:
import re

replaceDict = dict({
'{':" ", '}':" ", ',':"", '.':" ", '!':" ", '\\':" ", '/':" ", '$':" ", '%':" ",
'^':" ", '?':" ", '\'':" ", '"':" ", '(':" ", ')':" ", '*':" ", '+':" ", '-':" ",
'=':" ", ':':" ", ';':" ", ']':" ", '[':" ", '`':" ", '~':" ",
})

rep = dict((re.escape(k), v) for k, v in replaceDict.items())
pattern = re.compile("|".join(rep.keys()))
def replacer(text):
    return rep[re.escape(text.group(0))]

words = data.content.str.replace(pattern, replacer).str.lower().str.split()
words = pd.DataFrame(words.tolist())
words

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497
0,wanna,make,a,space,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,the,new,feature,“spaces”,is,causing,a,lot,of,arguments,and,tension,between,users,but,aside,from,that,it,is,a,great,app,where,people,can,communicate,and,make,friends,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,i,love,a,lot,of,twitter,account,i,spent,lots,of,time,on,twitter,and,its,make,me,happy,love,you,twitter,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,why,for,some,of,us,voice,notes,are,not,available,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,censorship,will,lead,to,dictatorship,even,if,you,think,you,understand,far,better,than,others,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,giant,waste,of,time,canceling,subscription,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1996,i,met,the,shmoop,of,my,life,on,this,app,10,10,recommend,but,sorry,i,took,the,only,shmoopy,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1997,completely,worthless,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1998,i,hate,it,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [17]:
#pip install --upgrade gensim

In [18]:
import gensim.downloader as model_api

word_vectors = model_api.load("glove-wiki-gigaword-50")

words.columns = words.columns.astype(str)

def soft_get(w):
    try:
        return word_vectors[w]
    except KeyError:
        return np.zeros(word_vectors.vector_size)
def map_vectors(row):
    try:
        return np.sum(
            row.loc[row.notna()].apply(soft_get)
        )
    except:
        return np.zeros(word_vectors.vector_size)
emb = pd.DataFrame(words.apply(map_vectors, axis=1))
emb.columns = ['C']
emb = pd.DataFrame(np.array(emb.C.apply(pd.Series)))
emb

[==================================================] 100.0% 66.0/66.0MB downloaded


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1.791390,1.228356,1.166130,0.019522,1.781600,-0.317260,-1.538457,-0.547340,0.037590,1.498080,-0.049464,0.744580,-0.480564,3.131580,1.668530,1.193420,1.861770,1.048700,-1.084350,-2.168500,0.230809,0.986180,-0.084390,0.549532,1.142400,-4.911380,-2.364010,0.348098,2.482940,-2.022670,11.547560,0.673800,-2.849750,0.754976,-0.104495,-0.003325,1.422772,0.495150,0.761970,-1.739460,0.234080,-0.230868,-0.308850,1.176802,-0.447965,-0.218770,0.717790,-1.179292,-0.772174,2.065610
1,13.803738,6.323400,-0.220026,1.512633,8.775153,8.528527,-9.190108,-5.127195,-2.939554,2.127037,-1.054101,1.058037,-3.998495,-5.753494,8.662167,6.778569,1.951510,1.723539,-5.985911,-12.867289,2.308367,4.270840,4.779436,5.249549,8.850356,-44.380536,-8.821725,1.252150,7.407345,-6.828607,97.580400,3.080823,-4.941615,-14.437838,0.268320,0.270213,-2.868169,0.859237,-0.733383,-2.158095,-0.894061,6.341992,-2.413470,6.754363,0.089843,5.623605,-1.199777,-3.092503,-2.719314,-2.570046
2,6.864309,7.461116,2.374861,-3.224091,9.240896,1.339739,-12.558339,-2.894539,-3.677803,5.402695,-5.189692,5.379338,-8.768253,-6.953367,16.274319,0.850833,-0.878141,2.171880,-1.419024,-9.279750,1.935139,8.841698,8.556992,4.402896,13.620778,-31.784267,-11.725950,1.313130,10.702894,-11.240800,68.949295,8.067888,-4.714981,-3.577744,-4.334715,-0.534639,-0.614536,2.177748,-0.936290,-6.814303,1.830049,-1.030380,-6.447449,5.357594,1.682907,-1.511144,-1.021652,-5.099684,-1.019366,3.594693
3,3.642210,2.392730,1.906705,-1.318636,3.852600,1.217293,-4.494641,-3.494194,-1.625258,1.106428,-0.932514,2.511110,0.065380,-1.382038,5.512600,1.903571,-0.691195,-1.050691,-0.549150,-6.462307,0.514977,2.417580,4.685011,0.458127,1.281240,-15.067450,-5.041889,-0.537628,1.384428,-3.736755,35.426399,1.425453,-0.924030,-3.951360,1.621320,-0.785574,0.484966,-1.290165,-1.446786,-2.032320,0.686342,2.358893,2.683360,3.029812,0.475740,1.558213,-2.199012,3.653490,-3.497716,1.069345
4,2.766726,-1.161190,4.070778,-3.928267,4.940870,-1.840893,-2.578837,-0.627970,-4.900071,4.506697,-1.258527,2.625170,-5.473810,-0.177783,10.233759,6.145055,5.449999,-2.044930,0.073049,-9.813700,-4.786398,3.856292,3.364767,0.152221,6.649774,-27.086599,-7.516039,-1.272255,8.268918,-5.303246,49.208500,7.039621,-6.682166,-5.624180,-3.584849,-2.994436,-0.241563,2.582508,0.440129,-4.990750,-4.266055,2.424954,3.970470,6.693950,0.273128,1.937435,-1.735125,0.219674,-2.478518,4.158286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.219280,1.227830,0.256887,1.554289,-0.679970,0.785914,-2.759544,-1.250050,2.414200,1.537637,0.221375,0.391189,0.289690,-0.038825,3.730196,0.678906,-0.611955,1.494630,-2.107470,-2.732174,4.879930,-1.870810,-0.600969,-1.093900,-0.440158,-5.901120,-0.986228,0.419670,1.501058,0.260583,14.754770,1.973921,-1.016780,0.397700,-2.085869,-0.425454,0.115267,0.589612,0.784320,-0.317670,0.348274,-1.400391,-0.079027,2.759642,-1.869002,0.080800,-1.333600,1.211950,-0.311883,-0.854720
1996,4.474093,3.884360,-0.289723,-2.687426,8.056744,0.349429,-7.725380,0.424527,-3.965073,-2.096396,-1.290270,2.076930,-8.174090,-3.357695,12.644321,2.439515,-2.485727,-0.792278,-6.279099,-4.534610,-0.216287,5.043168,7.209911,0.404685,6.224498,-29.299476,-8.089655,-0.310338,3.106572,-5.652033,58.196301,4.599459,-7.364157,-1.709973,1.984546,-2.383904,5.743566,3.559060,2.089139,-3.754350,-3.221640,0.183028,-1.535214,1.444747,-1.368396,2.905192,-2.342777,-2.637828,0.472709,2.196484
1997,0.697470,-2.387700,0.204993,-1.188000,0.672880,-0.151360,-0.271063,0.415820,-1.023540,0.632890,0.345043,0.833534,-0.109340,0.315618,1.430510,1.640500,0.313920,0.350022,0.967950,-0.047970,0.017410,0.224679,0.751660,0.049870,0.469690,-1.362600,-0.816030,2.002260,2.870800,-0.136101,3.804400,-1

In [19]:
emb = emb.fillna(0)

In [20]:
X = Normalizer().fit_transform(emb) #Normalizer gives better score
sgdr = SGDRegressor()
sgdr.fit(X, data['im:rating'])
data['gensim'] = sgdr.predict(X)
sgdr.score(X, data['im:rating'])

0.2461314572526777

In [21]:
pred = sgdr.predict(X)
r2_score(data['im:rating'], pred)

0.2461314572526777

In [22]:
# Question 3

data['app_version'] = data.App.astype(str) + '_' + data['im:version'].astype(str)
x = pd.get_dummies(data['app_version'], drop_first = True)

y = data['im:rating']
X = sm.add_constant(x)
est = sm.OLS(y, X).fit()

est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              im:rating   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     6.547
Date:                Sat, 13 Mar 2021   Prob (F-statistic):           6.51e-76
Time:                        00:59:25   Log-Likelihood:                -3420.0
No. Observations:                2000   AIC:                             7072.
Df Residuals:                    1884   BIC:                             7722.
Df Model:                         115                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.0000      1.378      3.627      0.000       2.297       7.703
Candy Crush_1.140.0      -1.2222      1.453     -0.841      0.400      -4.072       1.627
Candy Crush_1.142.0      -0.3846      1.430     -0.269      0.788      -3.190       2.421
Candy Crush_1.143.0      -0.8000      1.510     -0.530      0.596      -3.761       2.161
Candy Crush_1.144.0      -0.5000      1.688     -0.296      0.767      -3.811       2.811
Candy Crush_1.145.0      -2.6667      1.592     -1.675      0.094      -5.788       0.455
Candy Crush_1.146.1      -1.5000      1.489     -1.007      0.314      -4.420       1.420
Candy Crush_1.148.0    4.263e-13      1.949   2.19e-13      1.000      -3.823       3.823
Candy Crush_1.149.0    5.854e-13      1.949      3e-13      1.000      -3.823       3.823
Candy Crush_1.150.1    4.876e-13      1.688   2.89e-13      1.000      -3.311       3.311
Candy Crush_1.152.0      -0.5000      1.688     -0.296      0.767      -3.811       2.811
Candy Crush_1.153.0    4.272e-13      1.949   2.19e-13      1.000      -3.823       3.823
Candy Crush_1.154.0      -1.6667      1.592     -1.047      0.295      -4.788       1.455
Candy Crush_1.155.0      -0.6000      1.510     -0.397      0.691      -3.561       2.361
Candy Crush_1.160.0      -0.6667      1.592     -0.419      0.675      -3.788       2.455
Candy Crush_1.161.0      -1.0000      1.541     -0.649      0.517      -4.023       2.023
Candy Crush_1.162.1    4.676e-13      1.592   2.94e-13      1.000      -3.122       3.122
Candy Crush_1.164.0      -0.5000      1.688     -0.296      0.767      -3.811       2.811
Candy Crush_1.165.2      -2.0000      1.541     -1.298      0.195      -5.023       1.023
Candy Crush_1.166.0.4    -2.4000      1.510     -1.589      0.112      -5.361       0.561
Candy Crush_1.167.0.2  4.707e-13      1.949   2.41e-13      1.000      -3.823       3.823
Candy Crush_1.169.1.1    -4.0000      1.949     -2.052      0.040      -7.823      -0.177
Candy Crush_1.170.0.2  5.262e-13      1.688   3.12e-13      1.000      -3.311       3.311
Candy Crush_1.174.0.2    -3.0000      1.688     -1.777      0.076      -6.311       0.311
Candy Crush_1.175.0.4  4.854e-13      1.592   3.05e-13      1.000      -3.122       3.122
Candy Crush_1.176.0.2  5.185e-13      1.592   3.26e-13      1.000      -3.122       3.122
Candy Crush_1.180.0.1   4.61e-13      1.949   2.36e-13      1.000      -3.823       3.823
Candy Crush_1.184.1.2    -4.0000      1.949     -2.052      0.040      -7.823      -0.177
Candy Crush_1.186.0.3    -1.5000      1.541     -0.973      0.331      -4.523       1.523
Candy Crush_1.187.1.1    -3.0000      1.688     -1.777      0.076      -6.311       0.311
Candy Crush_1.188.0.4    -2.3846      1.430     -1.667      0.096      -5.190       0.421
Candy Crush_1.189.0.2  

In [23]:
#The R-squared is only 29.9%, meaning that the data is not very accurate. However, by merging the app column to its corresponding 
#versions, we can see which versions of each app had significant p-values. Facebook, Tinder, and Twitter had significantly different 
#ratings according to the p-values, while Candy Crush was more or less the same between each version.

In [24]:
#Question 4
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.58


In [29]:
fowlkes_mallows_score(y_pred, y_test)

0.5861482437130714

In [30]:
accuracy_score(y_pred, y_test)

0.58

In [32]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_pred, y_test, average = 'weighted')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8445891608391608, 0.58, 0.681984331430089, None)

In [ ]:
#Based on the results of the classification model, it is safe to assume that it is the best method for the ratings.
#The value from the classifier is 59%, whereas all previously calculated R-squared values were under 30%.
#It is interesting to note, however, that the accuracy score and Fowkes-Mallows for the TF-IDF were each around 65%.
#The classification model, however, yielded the same result of approximately 58% each time a metric was run.
#As such, the classification method may be better overall, but there is still some validity to TF-IDF.